In [21]:
import sys
sys.path.append('../../../build/python')

import mitsuba as mi
import drjit as dr
import tensorflow as tf

%load_ext autoreload
%autoreload 2
sys.path.append('../drjit')

from interop import wrap, to_drjit, from_drjit, flatten

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
# Test for a single tensor input

@wrap('tf', 'drjit')
def dr_func(input):
    return dr.power(input, 2)

x = tf.constant([1,2,3,4], tf.float32)
with tf.GradientTape() as tape:
    tape.watch(x)
    y = dr_func(x)
grad = tape.gradient(y, x)
print("x:", x)
print("y:", y)
print("Gradient:", grad)  


x: tf.Tensor([1. 2. 3. 4.], shape=(4,), dtype=float32)
y: tf.Tensor([ 1.  4.  9. 16.], shape=(4,), dtype=float32)
Gradient: tf.Tensor([2. 4. 6. 8.], shape=(4,), dtype=float32)


I0000 00:00:1728544523.904478   58436 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1728544523.906376   58436 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1728544523.945143   58436 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1728544523.947024   58436 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

In [44]:
# Test with a list of tensors

@wrap('tf', 'drjit')
def dr_func(inputs):
    result = 0
    for i, input in enumerate(inputs):
        result += dr.power(input, i+1)
    return result

x1 = tf.constant([1, 2, 3, 4], dtype=tf.float32)
x2 = tf.constant([1, 2, 3, 4], dtype=tf.float32)
x3 = tf.constant([1, 2, 3, 4], dtype=tf.float32)
vars = [x1, x2, x3]
with tf.GradientTape() as tape:
    tape.watch(vars)
    result = dr_func(vars)
grad = tape.gradient(result, vars)
print("Result:", result)
print("Gradient:")
for g in grad:
    print(g)

Result: tf.Tensor([ 3. 14. 39. 84.], shape=(4,), dtype=float32)
Gradient:
tf.Tensor([1. 1. 1. 1.], shape=(4,), dtype=float32)
tf.Tensor([2. 4. 6. 8.], shape=(4,), dtype=float32)
tf.Tensor([ 3. 12. 27. 48.], shape=(4,), dtype=float32)


In [43]:
def wrapper(func):
    def wrapper_2(*args, **kwargs):
        inputs = (args, kwargs)
        return func(*inputs)
    return wrapper_2

@wrapper
def func(*args, **kwargs):
    print(args)
    print(kwargs)
    return 1

x1 = tf.constant([1, 2, 3, 4], dtype=tf.float32)
x2 = tf.constant([1, 2, 3, 4], dtype=tf.float32)
x3 = tf.constant([1, 2, 3, 4], dtype=tf.float32)

func(x1, x2, {'x3': x3})

((<tf.Tensor: shape=(4,), dtype=float32, numpy=array([1., 2., 3., 4.], dtype=float32)>, <tf.Tensor: shape=(4,), dtype=float32, numpy=array([1., 2., 3., 4.], dtype=float32)>, {'x3': <tf.Tensor: shape=(4,), dtype=float32, numpy=array([1., 2., 3., 4.], dtype=float32)>}), {})
{}


1

In [45]:
# Test with a nested list of tensors
# Test with a list of tensors

@wrap('tf', 'drjit')
def dr_func(inputs):
    inputs = sum(inputs, [])
    result = 0
    for i, input in enumerate(inputs):
        result += dr.power(input, i+1)
    return result

x1 = tf.constant([1, 2, 3, 4], dtype=tf.float32)
x2 = tf.constant([1, 2, 3, 4], dtype=tf.float32)
x3 = tf.constant([1, 2, 3, 4], dtype=tf.float32)
x4 = tf.constant([1, 2, 3, 4], dtype=tf.float32)
vars = [[x1, x2], [x3, x4]]
with tf.GradientTape() as tape:
    tape.watch(vars)
    result = dr_func(vars)
grad = tape.gradient(result, vars)
print("Result:", result)
print("Gradient:")
for g in grad:
    print(g)


Result: tf.Tensor([  4.  30. 120. 340.], shape=(4,), dtype=float32)
Gradient:
[<tf.Tensor: shape=(4,), dtype=float32, numpy=array([1., 1., 1., 1.], dtype=float32)>, <tf.Tensor: shape=(4,), dtype=float32, numpy=array([2., 4., 6., 8.], dtype=float32)>]
[<tf.Tensor: shape=(4,), dtype=float32, numpy=array([ 3., 12., 27., 48.], dtype=float32)>, <tf.Tensor: shape=(4,), dtype=float32, numpy=array([  4.,  32., 108., 256.], dtype=float32)>]


In [20]:
def func(x):
    return x['x']

x = tf.constant(3, tf.float32)
z = tf.constant(4, tf.float32)

d = {
    'x' : x,
    'z' : z
}

func(d)

<tf.Tensor: shape=(), dtype=float32, numpy=3.0>

In [51]:
def wrapper(func):
    def wrapper_2(*args, **kwargs):
        #inputs = to_drjit((args, kwargs), 'tf', enable_grad=True)
        for i,x in enumerate(kwargs.values()):
            print(x)

        #inputs = to_drjit((args, kwargs), 'tf', enable_grad=True)
        #print(inputs)
        #outputs = func(*inputs)
        #results = from_drjit(outputs, 'tf')[0]
        #return results, grad
    return wrapper_2

@wrapper
def dr_func(*args, **kwargs):
    return True

x1 = tf.constant([1, 2, 3, 4], dtype=tf.float32)
x2 = tf.constant([2, 3, 4, 5], dtype=tf.float32)
x3 = tf.constant([3, 4, 5, 6], dtype=tf.float32)
result = dr_func(x1, x2, x3=x3)


tf.Tensor([3. 4. 5. 6.], shape=(4,), dtype=float32)


In [88]:
# Test with multiple arguments
@wrap('tf', 'drjit')
def dr_func(x1, x2):
    return dr.power(x1, 1) + dr.power(x2, 3)

x1 = tf.constant([1, 2, 3, 4], dtype=tf.float32)
x2 = tf.constant([1, 2, 3, 4], dtype=tf.float32)
with tf.GradientTape() as tape:
    tape.watch([x1, x2])
    result = dr_func(x1, x2)
grad = tape.gradient(result, [x1, x2])
print("Result:", result)
print("Gradient:")
for g in grad:
    print(g)


tf.Tensor([1. 1. 1. 1.], shape=(4,), dtype=float32)
Result: tf.Tensor([ 2. 10. 30. 68.], shape=(4,), dtype=float32)
Gradient:
tf.Tensor([1. 1. 1. 1.], shape=(4,), dtype=float32)
tf.Tensor([ 3. 12. 27. 48.], shape=(4,), dtype=float32)


In [55]:
def func(*args, **kwargs):
    print(args)
    print(kwargs)

func(1, 2, x=3)

(1, 2)
{'x': 3}


In [100]:
# Test with keyword argument
@wrap('tf', 'drjit')
def dr_func(*args, **kwargs):
    result = 0
    for i, x in enumerate(args):
        result += dr.power(x, i+1)

    for i, x in enumerate(kwargs.values()):
        result += 4*x

    return result

x1 = tf.constant([1, 2, 3, 4], dtype=tf.float32)
x2 = tf.constant([2, 3, 4, 5], dtype=tf.float32)
x3 = tf.constant([3, 4, 4, 5], dtype=tf.float32)

with tf.GradientTape() as tape:
    tape.watch([x1, x2, x3])
    result = dr_func(x1, x2, x3=x3)
grad = tape.gradient(result, [x1, x2, x3])
print("Result:", result)
print("Gradient:")
for g in grad:
    print(g)

ValueError: custom_gradient function expected to return 2 gradients, but returned 3 instead.

In [93]:
import tensorflow as tf

@tf.custom_gradient
def my_custom_function(*args, **kwargs):
    x, y = args
    scale = kwargs['scale']
    # Define the forward computation
    z = x * y * scale

    # Define the custom gradient function
    def grad(dz):
        # dz is the upstream gradient
        dx = dz * y * scale  # Gradient with respect to x
        dy = dz * x * scale  # Gradient with respect to y
        dscale = dz * x * y  # Gradient with respect to scale
        return dx, dy, dscale

    return z, grad

# Example usage
x = tf.constant(2.0)
y = tf.constant(3.0)
scale = tf.constant(0.5)

with tf.GradientTape() as tape:
    tape.watch([x, y, scale])
    result = my_custom_function(x, y, scale=scale)

# Compute gradients
grads = tape.gradient(result, [x, y, scale])
print("Result:", result.numpy())  # Forward computation result
print("Gradients:", [g.numpy() for g in grads])  # Gradients w.r.t x, y, and scale

Result: 3.0
Gradients: [1.5, 1.0, 6.0]
